https://platform.openai.com/docs/guides/code/best-practices 
Deprecated library, but instructions still helpful

* think about what information a programmer would need to perform a task. 
* clear comment or the data needed to write a useful function
* Specify the language 
* Specifying libraries
* Comment style can affect code quality. With some languages the style of comments can improve the quality of the output. For example, when working with Python, in some cases using doc strings (comments wrapped in triple quotes) can give higher quality results than using the pound (#) symbol.
* Prompt Codex with what you want it to do. (e.g. Create a web page, function)
* Put comments inside of functions can be helpful
* Provide examples for more precise results
* Limit completion size for more precise results or lower latenc


In [1]:
# https://platform.openai.com/examples/default-sql-translate
# rift on this idea ---> provide background information
# https://platform.openai.com/examples/default-function-from-spec
# rift on this idea ---> create python functions, etc. 

In [2]:
# Setup
import pandas as pd
df = pd.DataFrame({'name': ['Alice', 'Bob', 'Carol'], 'age': [25, 27, 29], 'grade': ['A', 'B', 'C']})
df2 = pd.DataFrame({'name': ['apple', 'banana', 'orange'], 'price': [1.2, 0.8, 0.6], 'quantity': [5, 3, 2]})
df3 = pd.DataFrame({'name': ['John', 'Mary', 'Bob'], 'department': ['Sales', 'Marketing', 'Engineering'], 'salary': [50000, 60000, 70000]})

In [3]:
import _wrapper as w

In [4]:
client = w._get_client()

In [13]:
prompt = r"""Create a plot in Python with matplotlib package.

Input data:


```python
# pandas DataFrame
'''
    name  age grade
0  Alice   25     A
1    Bob   27     B
2  Carol   29     C
'''
# DataFrame columns
'''
['name', 'age', 'grade']
'''

# pandas data frame variable is df
```
            


Plot should contain: make a scatter plot

Initial python code to be updated        

```python
# TODO import required dependencies
# TODO Provide the plot
```

Output only Python code."""

In [14]:
_ = w._get_response(client, user_content=prompt)

In [15]:
```python
import matplotlib.pyplot as plt

# Scatter plot
plt.scatter(df['age'], df['grade'])
plt.xlabel('Age')
plt.ylabel('Grade')

plt.show()
```

"```python\nimport matplotlib.pyplot as plt\n\n# Scatter plot\nplt.scatter(df['age'], df['grade'])\nplt.xlabel('Age')\nplt.ylabel('Grade')\n\nplt.show()\n```"

## Question 1 - Create code for altair?
Same command, replace matplotlib with altair

In [5]:
prompt=r"""Create a plot in Python with altair package. Output only Python code.

Input data:
```python
# pandas DataFrame
'''
    name  age grade
0  Alice   25     A
1    Bob   27     B
2  Carol   29     C
'''
# DataFrame columns
'''
['name', 'age', 'grade']
'''

# pandas data frame variable is df
```

Plot should contain: make a scatter plot

Initial python code to be updated        

```python
# TODO import required dependencies
# TODO Provide the plot
```

"""

In [6]:
_ = w._get_response(client, user_content=prompt)

In [16]:

# import required dependencies
import altair as alt

# dataframe
data = {'name': ['Alice', 'Bob', 'Carol'],
        'age': [25, 27, 29],
        'grade': ['A', 'B', 'C']}
df = pd.DataFrame(data)

# scatter plot
scatter_plot = alt.Chart(df).mark_point().encode(
    x='age:Q',
    y='grade:N',
    color='name:N'
).interactive()

# display the plot
#scatter_plot.show()
scatter_plot

alt.Chart(...)

## Experiment 2: Can I modify Altair prompt to get cleaner output?

In [38]:
prompt=r""" Return only the python script to complete the TODO
import altair as alt
import pandas as pd
data = {'name': ['Alice', 'Bob', 'Carol'],
        'age': [25, 27, 29],
        'grade': ['A', 'B', 'C']}

df = pd.DataFrame(data)
# Only return the following section
# TODO Complete the altair chart to make a scatter plot
"""

In [39]:
_ = w._get_response(client, user_content=prompt)

In [92]:
# TODO Complete the altair chart to make a scatter plot
chart = alt.Chart(df).mark_circle().encode(
    x='age',
    y='name',
    color='grade'
).interactive()

chart.show()

RuntimeError: Internal: _stream is not defined.

## Can I correct the usage via Examples

In [43]:
prompt=r"""
import altair as alt
import pandas as pd
data = {'name': ['Alice', 'Bob', 'Carol'],
        'age': [25, 27, 29],
        'grade': ['A', 'B', 'C']}

df = pd.DataFrame(data)
# Example altair plot - do not set to variable
# alt.Chart(df).mark_circle().encode(
#    x='age',
#    y='name',
#    color='grade'
# ).interactive()

# Only return the python code after todo
# TODO Complete the altair chart to make a bar plot

"""

In [44]:
_ = w._get_response(client, user_content=prompt)

In [45]:
# Completed altair chart

alt.Chart(df).mark_bar().encode(
    x='name',
    y='age',
    color='grade'
).interactive()

alt.Chart(...)

## Experiment - can I get tooltips & can I specify columns of interest

In [93]:
prompt=r"""
import altair as alt
import pandas as pd
data = {'name': ['Alice', 'Bob', 'Carol'],
        'age': [25, 27, 29],
        'grade': ['A', 'B', 'C']}

df = pd.DataFrame(data)
# Example altair plot - do not set to variable
# alt.Chart(df).mark_circle().encode(
#    x='age',
#    y='name',
#    color='grade'
# ).interactive()

# Only return the python code after todo
# TODO Complete the altair chart to make a bar plot with tooltips, use age, grade
"""

In [94]:
_ = w._get_response(client, user_content=prompt)
# attempt one dropped the color

In [95]:
alt.Chart(df).mark_bar().encode(
    x='age',
    y='grade',
    tooltip=['age', 'grade']
).interactive()

alt.Chart(...)

## Experiment - Infer the Altair Data Types
https://altair-viz.github.io/user_guide/data.html
https://altair-viz.github.io/user_guide/encodings/index.html#encoding-data-types

Can it reason with data provides

In [90]:
prompt=r"""
import altair as alt
import pandas as pd
data = {'name': ['Alice', 'Bob', 'Carol'],
        'age': [25, 27, 29],
        'grade': ['A', 'B', 'C']}

df = pd.DataFrame(data)

# Example altair plot - do not set to variable
alt.Chart(cars).mark_point().encode(
    alt.X('Acceleration', type='quantitative'),
    alt.Y('Miles_per_Gallon', type='quantitative'),
    alt.Color('Origin', type='nominal')
)).interactive()

# Only return the python code that creates Chart object
```python
# TODO Complete the altair chart that mirrors provided example
```
"""

In [91]:
_ = w._get_response(client, user_content=prompt) # 6 attempts
# I'm realizing the challenge here, need for experimentation and tracking


In [ ]:
```python
import altair as alt
import pandas as pd

data = {'name': ['Alice', 'Bob', 'Carol'],
        'age': [25, 27, 29],
        'grade': ['A', 'B', 'C']}

df = pd.DataFrame(data)

alt.Chart(df).mark_point().encode(
    alt.X('age', type='quantitative'),
    alt.Y('name', type='nominal'),
    alt.Color('grade', type='nominal')
).interactive()
```

In [71]:
prompt=r"""
```python
import altair as alt
import pandas as pd
data = {'name': ['Alice', 'Bob', 'Carol'],
        'age': [25, 27, 29],
        'grade': ['A', 'B', 'C']}

df = pd.DataFrame(data)
```
For each variable in data, provide one the Altair encoding data types (Q, O, N, T, G)
return only the results as a python dictionary
"""

In [73]:
_ = w._get_response(client, user_content=prompt)
# 2 pretty good, originally had name as ordered

In [ ]:
```python
encoding_types = {'name': 'N', 'age': 'Q', 'grade': 'N'}
```

In [98]:
prompt=r"""
```python
import altair as alt
import pandas as pd
data = {'name': ['Alice', 'Bob', 'Carol'],
        'age': [25, 27, 29],
        'grade': ['A', 'B', 'C'], 
        'dates': ['2023-01-01', '2023-02-01', '2023-03-01'] }

df = pd.DataFrame(data)
```
Only return python code:
Create a single python dictionary, where each variable is a key, and the value 
is one of the Altair encoding data types (Q, O, N, T, G)
"""

In [99]:
_ = w._get_response(client, user_content=prompt)
# see some of the challenges with this result

In [ ]:
```python
encoding = {
    'name': 'O',
    'age': 'N',
    'grade': 'N',
    'dates': 'T'
}
```

## Experiment - can I get leave generic, can it infer appropriate charts for data

In [104]:
prompt=r"""
```python
import altair as alt
import pandas as pd
data = {'name': ['Alice', 'Bob', 'Carol'],
        'age': [25, 27, 29],
        'grade': ['A', 'B', 'C'], 
        'dates': ['2023-01-01', '2023-02-01', '2023-03-01'] }

df = pd.DataFrame(data)
```
Recommend only one Altair mark_ for plotting name, age, grades based on data visualization best practices

"""

In [105]:
_ = w._get_response(client, user_content=prompt)

In [ ]:
Based on data visualization best practices, the most suitable mark for plotting name, age, and grades would be the bar mark. The bar mark is commonly used to show categorical data and provides a clear visual representation of comparisons between different categories. 

Here's an example of how to use the bar mark in Altair to plot the given data:

```python
import altair as alt

alt.Chart(df).mark_bar().encode(
    x='name',
    y='age',
    color='grade'
)
```

This will create a bar chart where the x-axis represents the "name" column, the y-axis represents the "age" column, and the bars are colored based on the "grade" column.

In [78]:
alt.Chart(df).mark_text().encode(
    x='name',
    y='age',
    text='grade',
    color='grade'
).properties(
    width=300
)

alt.Chart(...)

## Experiment - Can I create a compound chart?
https://altair-viz.github.io/user_guide/compound_charts.html#horizontal-concatenation

In [86]:
prompt=r"""
```python
import altair as alt
import pandas as pd
data = {'name': ['Alice', 'Bob', 'Carol'],
        'age': [25, 27, 29],
        'grade': ['A', 'B', 'C'], 
        'dates': ['2023-01-01', '2023-02-01', '2023-03-01'] }

df = pd.DataFrame(data)
```
return only python code, only return code related to Altair after the #TODO
#TODO: plot 2 charts, left chart uses name and age, right chart uses age and grade
"""

In [87]:
_ = w._get_response(client, user_content=prompt)
# 3 attempts - can't seem to figure out - only want 

```python
import altair as alt
import pandas as pd
data = {'name': ['Alice', 'Bob', 'Carol'],
        'age': [25, 27, 29],
        'grade': ['A', 'B', 'C'], 
        'dates': ['2023-01-01', '2023-02-01', '2023-03-01'] }

df = pd.DataFrame(data)

# TODO: plot 2 charts, left chart uses name and age, right chart uses age and grade

left_chart = alt.Chart(df).mark_circle().encode(
    x='name',
    y='age'
)

right_chart = alt.Chart(df).mark_circle().encode(
    x='age',
    y='grade'
)

(chart1 | chart2)
```

Note - code didn't work

In [89]:
left_chart = alt.Chart(df).mark_circle().encode(
    x='name',
    y='age'
)

right_chart = alt.Chart(df).mark_circle().encode(
    x='age',
    y='grade'
)

(left_chart | right_chart)

alt.HConcatChart(...)